In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import gc
import os
import seaborn as sns
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [2]:
application_train = pd.read_csv("HomeCredit_data/application_train.csv")
application_test = pd.read_csv("HomeCredit_data/application_test.csv")
application_train.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [3]:
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

In [4]:
def find_cats_non_cats(df_, grouper):
    
    df = df_.copy().drop(grouper, axis = 1)
    cats = [col for col in df.columns if df[col].dtype == "object"]
    non_cats = list(set(df.columns) - set(cats))
    return cats, non_cats
def aggregate_heuristics(df, grouper):
    cats_, non_cats_ = find_cats_non_cats(df, grouper)
    cats = cats_[:] + grouper
    non_cats = non_cats_[:] + grouper
    
    if not cats_:
        return (df.groupby(grouper).size().to_frame("size").
               merge(df[non_cats].groupby(grouper).mean(),
               left_index = True, right_index = True))
    if not non_cats_:
        return (df.groupby(grouper).size().to_frame("size").
               merge(df[cats].groupby(grouper).agg(lambda x: scipy.stats.mode(x)[0]),
               left_index = True, right_index = True))
    else:
        return (df.groupby(grouper).size().to_frame("size").
               merge(df[cats].groupby(grouper).agg(lambda x: scipy.stats.mode(x)[0]),
               left_index = True, right_index = True).
               merge(df[non_cats].groupby(grouper).mean(),
                    left_index = True, right_index = True))

In [5]:
import scipy

# Bureau Data
bureau = reduce_mem_usage(pd.read_csv("HomeCredit_data/bureau.csv"))
bureau_balance = reduce_mem_usage(pd.read_csv("HomeCredit_data/bureau_balance.csv"))

bureau_data = bureau.merge(aggregate_heuristics(bureau_balance, ['SK_ID_BUREAU']), left_on = 'SK_ID_BUREAU', right_index = True)
avg_bureau_data = aggregate_heuristics(bureau_data, ['SK_ID_CURR'])
app_train_bureau = application_train.merge(avg_bureau_data, left_on = 'SK_ID_CURR', right_index = True, how = 'left')
app_test_bureau = application_test.merge(avg_bureau_data, left_on = 'SK_ID_CURR', right_index = True, how = 'left')
del bureau_data, avg_bureau_data
gc.collect()

Memory usage of dataframe is 222.62 MB
Memory usage after optimization is: 119.49 MB
Decreased by 46.3%
Memory usage of dataframe is 624.85 MB
Memory usage after optimization is: 338.46 MB
Decreased by 45.8%


D:\Users\850024011\AppData\Local\Continuum\anaconda3\lib\site-packages\scipy\stats\stats.py:245: RuntimeWarning: The input array could not be properly checked for nan values. nan values will be ignored.
  "values. nan values will be ignored.", RuntimeWarning)


28

In [6]:
# Previous credits Data
credit_card_balance = reduce_mem_usage(pd.read_csv("HomeCredit_data/credit_card_balance.csv"))
avg_credi_card = aggregate_heuristics(credit_card_balance, ['SK_ID_PREV', 'SK_ID_CURR'])
del credit_card_balance
gc.collect()

Memory usage of dataframe is 673.88 MB
Memory usage after optimization is: 318.63 MB
Decreased by 52.7%


D:\Users\850024011\AppData\Local\Continuum\anaconda3\lib\site-packages\scipy\stats\stats.py:245: RuntimeWarning: The input array could not be properly checked for nan values. nan values will be ignored.
  "values. nan values will be ignored.", RuntimeWarning)


126

In [7]:
#Installment payments
installments_payments = reduce_mem_usage(pd.read_csv("HomeCredit_data/installments_payments.csv"))
avg_installment = aggregate_heuristics(installments_payments, ['SK_ID_PREV', 'SK_ID_CURR'])
del installments_payments
gc.collect()

Memory usage of dataframe is 830.41 MB
Memory usage after optimization is: 389.25 MB
Decreased by 53.1%


91

In [8]:
previous_application = reduce_mem_usage(pd.read_csv("HomeCredit_data/previous_application.csv"))
avg_previous = aggregate_heuristics(previous_application, ['SK_ID_PREV', 'SK_ID_CURR'])
avg_previous.head()

Memory usage of dataframe is 471.48 MB
Memory usage after optimization is: 324.94 MB
Decreased by 31.1%


D:\Users\850024011\AppData\Local\Continuum\anaconda3\lib\site-packages\scipy\stats\stats.py:245: RuntimeWarning: The input array could not be properly checked for nan values. nan values will be ignored.
  "values. nan values will be ignored.", RuntimeWarning)


,,size,NAME_CONTRACT_TYPE,WEEKDAY_APPR_PROCESS_START,FLAG_LAST_APPL_PER_CONTRACT,NAME_CASH_LOAN_PURPOSE,NAME_CONTRACT_STATUS,NAME_PAYMENT_TYPE,CODE_REJECT_REASON,NAME_TYPE_SUITE,NAME_CLIENT_TYPE,...,NFLAG_LAST_APPL_IN_DAY,NFLAG_INSURED_ON_APPROVAL,DAYS_LAST_DUE,RATE_INTEREST_PRIVILEGED,AMT_CREDIT,AMT_ANNUITY,AMT_APPLICATION,SELLERPLACE_AREA,DAYS_TERMINATION,DAYS_LAST_DUE_1ST_VERSION
SK_ID_PREV,SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
1000001,158271,1,Consumer loans,THURSDAY,Y,XAP,Approved,Cash through the bank,XAP,0,New,...,1,0.0,-238.0,NaN,65124.0,6404.310059,58905.000000,70,-233.0,62.0
1000002,101962,1,Consumer loans,SUNDAY,Y,XAP,Approved,Cash through the bank,XAP,Unaccompanied,Repeater,...,1,0.0,-1510.0,NaN,35230.5,6264.000000,39145.500000,1780,-1501.0,-1450.0
1000003,252457,1,Consumer loans,SUNDAY,Y,XAP,Approved,Cash through the bank,XAP,Unaccompanied,Refreshed,...,1,1.0,365243.0,NaN,52641.0,4951.350098,47056.273438,1103,365243.0,236.0
1000004,260094,1,Consumer loans,THURSDAY,Y,XAP,Approved,XNA,XAP,Unaccompanied,Refreshed,...,1,0.0,-682.0,NaN,30586.5,3391.110107,35144.371094,1103,-672.0,-592.0
1000005,176456,1,Consumer loans,THURSDAY,Y,XAP,Approved,Cash through the bank,XAP,Family,New,...,1,0.0,-1418.0,NaN,120307.5,14713.605469,123486.078125,1877,-1415.0,-1418.0


In [9]:
POS_CASH_balance = reduce_mem_usage(pd.read_csv("HomeCredit_data/POS_CASH_balance.csv"))
avg_cash_balance = aggregate_heuristics(POS_CASH_balance, ['SK_ID_PREV', 'SK_ID_CURR'])
del previous_application, POS_CASH_balance
gc.collect()

Memory usage of dataframe is 610.43 MB
Memory usage after optimization is: 276.60 MB
Decreased by 54.7%


D:\Users\850024011\AppData\Local\Continuum\anaconda3\lib\site-packages\scipy\stats\stats.py:245: RuntimeWarning: The input array could not be properly checked for nan values. nan values will be ignored.
  "values. nan values will be ignored.", RuntimeWarning)


77

In [10]:
previous_final = avg_installment.merge(avg_credi_card, left_index = True, right_index = True, how = 'outer', suffixes = ['_installment','_credit']).merge(avg_cash_balance, left_index = True, right_index = True, how = 'outer', suffixes = ['__','_balance']).merge(avg_previous, left_index = True, right_index = True, how = 'outer', suffixes = [':__','_previous'])
previous_final.head()

,,size_installment,DAYS_INSTALMENT,NUM_INSTALMENT_NUMBER,AMT_INSTALMENT,AMT_PAYMENT,NUM_INSTALMENT_VERSION,DAYS_ENTRY_PAYMENT,size_credit,NAME_CONTRACT_STATUS__,AMT_INST_MIN_REGULARITY,...,NFLAG_LAST_APPL_IN_DAY,NFLAG_INSURED_ON_APPROVAL,DAYS_LAST_DUE,RATE_INTEREST_PRIVILEGED,AMT_CREDIT,AMT_ANNUITY,AMT_APPLICATION,SELLERPLACE_AREA,DAYS_TERMINATION,DAYS_LAST_DUE_1ST_VERSION
SK_ID_PREV,SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
1000001,158271,2.0,-253.00000,1.500000,34221.710938,34221.710938,1.500000,-269.000000,NaN,NaN,NaN,...,1.0,0.0,-238.0,NaN,65124.0,6404.310059,58905.000000,70.0,-233.0,62.0
1000002,101962,4.0,-1555.00000,2.500000,9308.890625,9308.890625,1.250000,-1574.750000,NaN,NaN,NaN,...,1.0,0.0,-1510.0,NaN,35230.5,6264.000000,39145.500000,1780.0,-1501.0,-1450.0
1000003,252457,3.0,-64.00000,2.000000,4951.350098,4951.350098,1.000000,-79.333336,NaN,NaN,NaN,...,1.0,1.0,365243.0,NaN,52641.0,4951.350098,47056.273438,1103.0,365243.0,236.0
1000004,260094,7.0,-772.00000,4.000000,4789.022461,4789.022461,1.142857,-798.714294,NaN,NaN,NaN,...,1.0,0.0,-682.0,NaN,30586.5,3391.110107,35144.371094,1103.0,-672.0,-592.0
1000005,176456,11.0,-1543.45459,5.818182,14703.209961,13365.610352,1.000000,-1551.909058,NaN,NaN,NaN,...,1.0,0.0,-1418.0,NaN,120307.5,14713.605469,123486.078125,1877.0,-1415.0,-1418.0


In [11]:
app_bureau_previous = pd.merge(app_train_bureau, previous_final, on = 'SK_ID_CURR', how = 'left')
app_test_bureau_previous = pd.merge(app_test_bureau, previous_final, on = 'SK_ID_CURR', how = 'left')
del previous_final
gc.collect()

7

In [12]:
app_bureau_previous.dtypes.value_counts()

float64    90
float32    45
int64      41
object     38
dtype: int64

In [13]:
app_test_bureau_previous.dtypes.value_counts()

float64    90
float32    45
int64      40
object     38
dtype: int64

In [14]:
#Save the merger train and test data
app_bureau_previous.to_csv('MergedData/merged_data_train.csv')
app_test_bureau_previous.to_csv('MergedData/merged_data_test.csv')